In [ ]:
import pandas as pd

import numpy as np

import nltk
from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

from nltk.sentiment import vader
from nltk.sentiment.vader import SentimentIntensityAnalyzer

from nltk.tokenize import sent_tokenize
from nltk import word_tokenize
from nltk.corpus import stopwords

import sklearn
from sklearn.feature_extraction.text import CountVectorizer

from sklearn.metrics import classification_report


In [ ]:
filename ="train.csv"
df = pd.read_csv(filename, encoding = 'ISO-8859-1')

df = df[['textID', 'text', 'sentiment']]
df = df.dropna()
df.head()


# 
# rows_per_sent = 1000  # each sentiment has this number of rows
# df_pos = df[df['sentiment']=='positive']
# df_pos = df_pos.head(rows_per_sent)
# df_neut = df[df['sentiment']=='neutral']
# df_neut = df_neut.head(rows_per_sent)
# df_neg = df[df['sentiment']=='negative']
# df_neg = df_neg.head(rows_per_sent)
# df_neg

# df = pd.concat([df_pos,df_neut, df_neg])

# print(len(df['sentiment']))
# df['sentiment'].value_counts()

textID                                               text sentiment
0  cb774db0d1                I`d have responded, if I were going   neutral
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!  negative
2  088c60f138                          my boss is bullying me...  negative
3  9642c003ef                     what interview! leave me alone  negative
4  358bd9e861   Sons of ****, why couldn`t they put them on t...  negative

In [ ]:
df.head()

# preprocessing

In [ ]:
#import nltk
nltk.download('punkt')
nltk.download('vader_lexicon')
nltk.download('averaged_perceptron_tagger')
nltk.download('maxent_ne_chunker')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!


True

In [ ]:
# tokenize 
tokens_per_sentence = []
for sentence in df['text']:
    sent_tokens = word_tokenize(sentence)
    tokens_per_sentence.append(sent_tokens)

In [ ]:
sent_id = 2
print('SENTENCE', df['text'][sent_id])
print('TOKENS', tokens_per_sentence[sent_id])

SENTENCE my boss is bullying me...
TOKENS ['my', 'boss', 'is', 'bullying', 'me', '...']


In [ ]:
# POS-tagging

pos_tags_per_sentence = []
for tokens in tokens_per_sentence:
    sentence = nltk.pos_tag(tokens)
    pos_tags_per_sentence.append(sentence)

# check output if tagging is correct.
#for sent in pos_tags_per_sentence: print (sent)

# Sentiment Analysis

## VADER

In [ ]:
# dic = vader_model.polarity_scores(sentence)
def sentimentClassifier(dic):
    global vader_model
    neg = dic['neg']
    neu = dic['neu']
    pos = dic['pos']
    
    sentiment_scores = [neg,neu,pos]
    sentiment_scores_name = ['negative', 'neutral', 'positive']
    
    sentiment_scores_argmax = np.argmax(sentiment_scores)
    prediction = sentiment_scores_name[sentiment_scores_argmax]
    return prediction


In [ ]:
vader_model = SentimentIntensityAnalyzer()

In [ ]:
# genereert rows en scores. ik weet niet waar ik die moet opslaan
scores_per_row = []
for row in df['text']:
    scores = vader_model.polarity_scores(row)
    scores_per_row.append(scores)


In [ ]:
sent_id = 1
print('SENTENCE', df['Review'][sent_id])
print('TOKENS', tokens_per_sentence[sent_id])


In [ ]:
sentimentClassifier(vader_model.polarity_scores(text))

## sentiment evaluation

## data preparation

In [ ]:
# test set
SA_test = 'sentiment-topic-final-test.tsv'
SA_test = pd.read_csv(SA_test,sep='\t')
SA_test.head()

sentence id                                               text sentiment  \
0            0  It took eight years for Warner Brothers to rec...  negative   
1            1  All the New York University students love this...  positive   
2            2  This Italian place is really trendy but they h...  negative   
3            3  In conclusion, my review of this book would be...  positive   
4            4  The story of this movie is focused on Carl Bra...   neutral   

        topic  
0       movie  
1  restaurant  
2  restaurant  
3        book  
4       movie

In [ ]:
def vectorizer(min_df_input):
    airline_vec = CountVectorizer(min_df= min_df_input, # If a token appears fewer times than this, across all documents, it will be ignored
                             tokenizer=nltk.word_tokenize, # we use the nltk tokenizer
                             stop_words=stopwords.words('english')) # stopwords are removed
    return airline_vec


In [ ]:
# vectorize the data
nltk.download('stopwords')
vect = vectorizer(5)
df_text_vec_fit_trans = vect.fit_transform(df['text'])

SA_test_vec_trans = vect.transform(SA_test['text'])

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ["'d", "'ll", "'re", "'s", "'ve", 'could', 'might', 'must', "n't", 'need', 'sha', 'wo', 'would'] not in stop_words.
  warnings.warn(


## support vector machine

In [ ]:
from sklearn import svm

# Perform classification with SVM, kernel=linear
classifier_linear = svm.SVC(kernel='linear')

classifier_linear.fit(df_text_vec_fit_trans, df['sentiment'])

prediction_linear = classifier_linear.predict(SA_test_vec_trans)

# results
report = classification_report(SA_test['sentiment'], prediction_linear,target_names=SA_test['sentiment'].unique())
print(report)


In [ ]:
# dic = vader_model.polarity_scores(sentence)
def sentimentClassifier(dic):
    global vader_model
    neg = dic['neg']
    neu = dic['neu']
    pos = dic['pos']
    
    sentiment_scores = [neg,neu,pos]
    sentiment_scores_name = ['negative', 'neutral', 'positive']
    
    sentiment_scores_argmax = np.argmax(sentiment_scores)
    prediction = sentiment_scores_name[sentiment_scores_argmax]
    return prediction


vader_model = SentimentIntensityAnalyzer()

SA_test = 'sentiment-topic-final-test.tsv'
SA_test = pd.read_csv(SA_test,sep='\t')

scores_per_row = []
for row in SA_test['text']:
    scores = vader_model.polarity_scores(row)
    sentiment = sentimentClassifier(scores)
    scores_per_row.append(sentiment)
print(scores_per_row)

test_sentiment = SA_test['sentiment'].tolist()



['neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral', 'neutral']


'              precision    recall  f1-score   support\n\n    negative       1.00      0.00      0.00         3\n     neutral       0.30      1.00      0.46         3\n    positive       1.00      0.00      0.00         4\n\n    accuracy                           0.30        10\n   macro avg       0.77      0.33      0.15        10\nweighted avg       0.79      0.30      0.14        10\n'

In [ ]:
print(classification_report(test_sentiment, scores_per_row, zero_division=True))

              precision    recall  f1-score   support

    negative       1.00      0.00      0.00         3
     neutral       0.30      1.00      0.46         3
    positive       1.00      0.00      0.00         4

    accuracy                           0.30        10
   macro avg       0.77      0.33      0.15        10
weighted avg       0.79      0.30      0.14        10

